#### 元类编程

In [ ]:
# 8-1 property动态属性    Python内置的@property装饰器就是负责把一个方法变成属性调用的
'''
@property可以将python定义的函数“当做”属性访问，从而提供更加友好访问方式，但是有时候setter/deleter也是需要的。
1》只有@property表示只读。
2》同时有@property和@x.setter表示可读可写。
3》同时有@property和@x.setter和@x.deleter表示可读可写可删除。
@property 装饰器实现了对类属性的控制和保护 
@property 装饰器兼顾了方便和控制，让实例的使用显得更加优雅，提高了可用性。另外，@property装饰器不设置setter时，就是一个只读属性，相当于对属性起到了保护作用  https://www.cnblogs.com/bitterz/p/10237602.html
'''

In [10]:
from datetime import date, datetime     # datetime库介绍   https://www.jianshu.com/p/82008ba6706e
class User:
    def __init__(self, name, birthday):
        self.name = name
        self.birthday = birthday
        self._age = 0

    def get_age(self):
        return datetime.now().year - self.birthday.year

if __name__ == "__main__":
    user = User("bobby", date(year=1987, month=1, day=1))
    print(user.get_age())
    


32


In [3]:
from datetime import date, datetime 
class User:
    def __init__(self, name, birthday):
        self.name = name
        self.birthday = birthday
        self._age = 0   # 加上单下划线，实际上只是一种编程规范，意思是说我们不想把这个属性暴露出去，只能通过类的方法进行调用，实际上在Python中，只有双下划线__才有此作用

    '''
    假设原本有一个age属性，我们可以使用user.age的方式调用，但是后来我们去除了该属性，
    就不能使用user.age的方式了，此时我们可以定义一个叫age的方法，然后使用@property修饰，使该方法变成一个可以被调用的属性
    '''
    @property     # 相当于一个取的操作了 get     使用@property 使得一个函数可以当做属性进行调用   
    def age(self):  
        return datetime.now().year - self.birthday.year
    #@property 装饰器兼顾了方便和控制，让实例的使用显得更加优雅，提高了可用性。另外，@property装饰器不设置setter时，就是一个只读属性，相当于对属性起到了保护作用
    
    @age.setter   # @property本身又创建了另一个装饰器@age.setter，负责把一个setter方法变成属性赋值，于是，我们就拥有一个可控的属性操作：
    def age(self, value):
        self._age = value
    
    def print_age(self):
        print(self._age)
        
if __name__ == "__main__":
    user = User("bobby", date(year=1987, month=1, day=1))
    user.age = 30  # 调用了 @age.setter 修饰的方法
    user.print_age()
    print(user.age)
    
    
    print(type(user.age))  # 因为装饰器函数在被装饰函数定义好后立即执行，这里得到的就是返回值的类型 int 了

30
32
<class 'int'>


In [ ]:
说明：

1. @property相当于对age方法进行了一个装饰，它使得我们能通过 对象.方法名 来调用对应的属性

2. @property所装饰的方法名与 对象. 调用的名称要保持一致

3. @property会生成另外的装饰器，@方法名.setter, @方法名.getter, @方法名.deleter，分别对应set, get, del方法。这里get方法用得很少，因为已经通过@property直接对应到了get方法
使用这些装饰器装饰的方法都是用的同样的名字 方法名

@方法名.setter 修饰的方法由于需要接受一个被设置的参数值，所以需要有一个value参数，其余的方法都不需要参数
使用property属性托管的功能用法的时候，方法的参数也是一样

##### `property`使用说明       `property` 不适合大规模使用

In [92]:
# property 的使用说明  典型的用法是托管属性            看一看描述器协议的内容，因为实际上 property托管属性的功能实现原理就是基于描述器协议的__get__方法（具体在描述器协议中看），而装饰器@property更是如此（这个可以参照1-4中的装饰器部分例二） 
class C(object): 
            def getx(self): return self._x
            def setx(self, value): self._x = value
            def delx(self): del self._x
            x = property(getx, setx, delx, "I'm the 'x' property.")   
            '''
            该语法是利用描述器协议实现的，因为在property类中，实现了__get__(self, instance, type)、__set__(self, instance, value)、__delete__(self, instance)这三个方方，所以成了描述符
            '''

# @property说明 装饰器可以轻松定义新属性或修改原有属性
class C(object):
            @property
            def x(self):
                "I am the 'x' property."
                return self._x
            @x.setter
            def x(self, value):
                self._x = value
            @x.deleter
            def x(self):
                del self._x

##### `property`之托管属性用法

In [22]:
# 托管属性 用法示例 1 
class C(object):
    def getx(self): return self._x
    def setx(self, value): self._x = value
    def delx(self): del self._x 
    x = property(getx, setx, delx, "I'm the 'x' property.")   # 声明一个property实例对象 

c = C()  # 想使用被托管的属性，一定要创建一个类实例来调用
c.x =1   # 此例中，一定要先动态创建一个变量才可以使用getx的语句，所以先用 c.x 调用setx创建一个变量         
print(c.x) 
print(C.x.__doc__) # 大写的C
#  property(self, fget=None, fset=None, fdel=None, doc=None) 中有第四个属性doc，是对当前property的说明，想打印该参数，必须使用类来调用property的实例对象x，因为此时x才是property实例对象，
# 而使用类实例调用x的话就已经开始使用property的功能了，其次必须使用__doc__方能打印property的doc内容，因为property没有属性doc，是在doc接受到值的时候，动态修改property类的__doc__的内容（即文档说明）
# 使用property.__doc__任然能查到文档内容，但是一般情况下我们使用类的实例对象调用__doc__也是能打印类的文档内容的，但是property做了处理，动态改变了其实例调用__doc__得到的说明文档内容

# 解释： 通过 dir(类实例对象) 我们可以发现，类对象是可以调用类的__doc__方法的，该方法打印类的注释文档，所以使用类实例对象.__doc__打印的是类的文档，而我们这里print(C.x.__doc__)  本应该打印的是property的说明文档
# 而这里实际是动态的改变的了打印的内容

1
I'm the 'x' property.


In [8]:
# 托管属性 用法示例 2
'''
三个参数分别是，获取属性的方法，设置属性的方法和删除属性的方法
作用是通过属性来设置属性
使用的好处：对于类中有多个方法时候，之后要对这些方法进行大改的时候，可能会对之前已经调用的地方进行大量的修改，有了property()则只对使用者提供了x这一个接口，无论是用户使用，还是程序员修改都是方便的。
'''
class C(object):
    def __init__(self, size=10):self.size = size 
    def getSize(self): return self.size
    def setSize(self, value): self.size = value
    def delSize(self): 
        del self.size 
        # 注意这里是删除实例对象的一个属性size，不是当前类的实例对象，是不是调用当前类定义的__del__魔法方法的
        # 第一：与__del__无关，不会调用类的__del__魔法函数，只有删除  实例对象  的  引用   ，使其引用数为0才会调用__del__方法
        # 其次：这里与什么引用数也无关，它删除的是一个  类属性  （删除的是类实例的属性，而不是一个实例类实例对象），相当于一个指针 说明看下面的 
        print("实例对象的size属性被删除")                 
    x = property(getSize, setSize, delSize)   # 由property类的实现，这里的getSize,setSize，delSize三个函数名参数传递给了property类的__get__(self, instance, type)、__set__(self, instance, value)、__delete__(self, instance)三个方法

c1 = C()
print(c1.x)
c1.x = 20
print(c1.x)
del c1.x
# print(c1.x)  # AttributeError: 'C' object has no attribute 'size'




10
20
实例对象的size属性被删除


In [88]:
#  关于垃圾回收问题 看第五章 # 7-3 del语句和垃圾回收

'''
  python 中的变量名有些类似于C++中的指针的概念.
  python 中的赋值操作改变的并不是内存中变量的值, 而是变量名指向的变量.
'''

a = "hello"
b = a       # b也是指向"hello"对象
print(id(a))
print(id(b))
print("以上两次的 id 都一样，说明指向的是同一个对象")
print(id("hello"))    # intern机制  https://www.cnblogs.com/greatfish/p/6045088.html
'''
此处使用 id('hello') 打印的时候，理论上应该是不同的地址，因为这里实际上是生成的新的对象（即使名字是一样的，与a = 'hello' 不是同一个'hello'对象），造成这样原因是因为Python处理字符串时的intern（字符串驻留）机制，
会将已有的字符串即值同样的字符串对象仅仅会保存一份，放在一个字符串储蓄池中，是共用的
'''
del a
print(b)    # 只是a指针被删除了,"hello"的引用数减少1，当没有为0，所以"hello"对象还在


# 上边的例子指出, 当执行 " a = 'hello' " 时, python会在内存中分配一块空间, 把'hello'赋值给这块空间.
# 然后, 将变量名 "a" 关联到'hello'分配的内存.
# 当执行 "b=a" 时,同样将变量名 "b" 关联到'hello'分配的内存.'hello'有两份引用
# 所以删除 a指针后，'hello'对象任然存在，调用b人得到'hello' ---- （这里我们不讨论intern（字符串驻留）机制导致的'hello'对象 有其他的引用）.  ---  对没有intern（字符串驻留）机制的其他类型，上面的理论完全有效， 如长整型 1111111111111111 

2309323515976
2309323515976
以上两次的 id 都一样，说明指向的是同一个对象
2309323515976
hello


##### `@property`装饰器使用说明

In [9]:
# https://www.cnblogs.com/bitterz/p/10237602.html
# https://www.cnblogs.com/Meanwey/p/9902668.html

# 使用@property的好处是我们可以对我们的内置属性进行检测，限制  --  因为我们是通过函数去操作属性的
# 注意 @property def score 和 @score.setter 是一套完整的 设置一个可读可写的变量 

# 示例：在绑定属性时，如果我们直接把属性暴露出去，虽然写起来很简单，但是，没办法检查参数，导致可以把成绩随便改
class Student(object):
    
    @property
    def score(self):                
        return self._score
    # score( )方法在使用“@property”装饰器之后已经成为类中的普通属性了，会与原本的score属性冲突（即之后调用的score都是由@property产生的，原先类定义的score将无法调用），因此建议读者平时写代码时养成良好的习惯，在定义类属性时使用下划线作为前缀。
    
    @score.setter
    def score(self, value):
        if not isinstance(value, int):
            raise ValueError("成绩必须为整数")
        if value < 0 or value > 100:
            raise ValueError("成绩必须在 0 到 100 之间！")
        self._score = value   # 动态生成实例属性
        
s = Student()
print(s.__dict__)

s.score = 90   # 优先调用该方法，动态生成实例属性
print("成绩：", s.score)

print(s.__dict__)  # 发现此时动态添加了_score实例属性，这是因为类实例的属性是可以在类的内部方法中动态添加的

s.score = -100
print("成绩：", s.score)


{}
成绩： 90
{'_score': 90}


ValueError: 成绩必须在 0 到 100 之间！

In [58]:
class a:
    def pp(self,value):
        self.a =value
    
b = a()
print(b.__dict__)
b.pp(2)            # 实例属性是可以动态生成的
print(b.__dict__)
b.a

{}
{'a': 2}


2

In [4]:
# 请利用@property给一个Screen对象加上width和height属性，以及一个只读属性resolution：

class Screen():
    @property
    def width(self):
        return self._width

    @width.setter
    def width(self, value):
        self._width = value

    @property
    def height(self):
        return self._height

    @height.setter
    def height(self, value):
        self._height = value
  
    @property
    def resolution(self):
        return self._height * self._width


if __name__ == "__main__":
    s = Screen()
    s.width = 1024
    s.height = 768
    print('resolution =', s.resolution)
    if s.resolution == 786432:
        print('测试通过!')
    else:
        print('测试失败!')

    print(s.__dict__)


resolution = 786432
测试通过!
{'_width': 1024, '_height': 768}


In [95]:
# 8-2 __getattr__、__getattribute__魔法函数    调用属性 和 方法的时候用到
# __getattr__ 就是在查找不到属性的时候调用      
from datetime import date
class User:
    def __init__(self,info={}):
        self.info = info

if __name__ == "__main__":
    user = User(info={"company_name":"imooc", "name":"bobby"})
    print(user.test)


AttributeError: 'User' object has no attribute 'test'

<font color='red' size='6'>注意：`__getattribute__ `和 `__getattr__` 都必须要有返回值  即要有return 语句  必须</font>

In [104]:
from datetime import date
class User:
    def __init__(self,info={}):
        self.info = info

    def __getattr__(self, item):    # 当查找的属性不存在的时候，就会调用__getattr__魔法函数 
        return self.info[item]  

if __name__ == "__main__":
    user = User(info={"company_name":"imooc", "name":"bobby"})
    print(user.name)   

bobby


In [113]:
# __getattr__接收一个不存在的方法，此时__getattr__要求也要返回一个方法，且此方法还要求能够接受不存在的方法的参数，__getattribute__同理
class Test(object):
    def __getattr__(self, value):
        print(value)
        print(type(value))
        return len                   # 可见，接受的是方法名，且return的方法一定要能接受不存在的方法的参数

test = Test()
test.getlength("dfsaf")

getlength
<class 'str'>


5

In [114]:
# __getattribute__魔法函数，是当只要调用属性，或是调用方法都会调用该方法    
'''
实际上，__getattr__是在__getattribute__中被的调用的   
没有必要的时候，尽量不要重写这个方法，因为一旦写不好就会使整个类的属性方法崩溃掉，---  所以在重写的时候，一般要在其内部继承父类的该方法
一般在写框架的时候重写这个方法，用于控制整个类实例的过程以及类属性访问的过程

观点一：可以看下面的131 因为类的实例方法是在类创建完的时候就创建完成的，而实际属性则是调用__new__(cls)创建实例对象，然后会调用__init__创建实例的属性，
此时就要调用__setattr__(self, key, value)，在其中调用self.__dict__是就会使用到__getattribute__，此时如果对__getattribute__进行了重写了，且没有继承父类的__getattribute__，可能使整个类的属性方法崩溃掉
无法创建__dict__，见下例子
 

'''
class Test(object):
    def __init__(self, name):
        self.name = name
        
    def __getattribute__(self, value):
        if value == 'address':
            return "China"
        return super().__getattribute__(value)  # 实现定制化，继承父类，一般值可以正常返回
    
if __name__ == "__main__":
    test = Test("ietian")
    print(test.name)
    print(test.address)
    test.address = "Anhui"   # 这里动态给当前实例对象创建了属性address，依然被过滤了 
    print(test.address)
    

ietian
China
China


In [32]:
class Test(object):
    def __init__(self, name):
        self.name = name
        
    def __getattribute__(self, value):
        print("called")
        return super().__getattribute__(value)  # 实现定制化，继承父类，一般值可以正常返回
    
    def __setattr__(self,key ,value): 
        '''
        因为self.__dict__是调用了实例属性，所以会调用了__getattribute__方法    但是没有重写__setattr__的时候，没有调用__getattribute__方法      
        
        问题： 那么没有重写__setattr__时候，类实例__dict__属性从哪里来呢??而__setattr__又是怎样进行初始化的呢？？？？
        
        猜测是从基类object出继承过来的，因为object在定义的时候，创建了类属性 __dict__
        '''
        self.__dict__[key] = value   
                                    
    
if __name__ == "__main__":
    test = Test("ietian")


    

called


<font color = "red" size = 6> 一般的，除非是自己定义一个框架，否则是不会重写`__getattribute__(self, itme)`的（重写的时候为了保证实例属性的正常创建，一般还要继承父类的同类方法），但是会重写`__getattr__(self, item)`(当要调用的实例方法或者属性不存在的时候，会调用这个方法)，一般也不会重写`__setattr__(self, key, value)`方法，因为__init__时候会调用此方法，对`__setattr__`的修改也会影响到实例属性的初始化 </font></br>
</br>
<font color = "red" size = 6>另外要注意的一点就是：在`__setattr__`中创建实例属性的时候，是一定要通过`self.__dict__`调用内置的`__dict__`实例属性字典的形式去进行赋值，而不能使用`self.key = value`的形式去使用，因为因为__setattr__ </font>

In [208]:
# __getattr__ 和 __getattribute__ 关系
class AboutAttr(object):
    def __init__(self, name):
        self.name = name
        
    def __getattribute__(self, value):
        try:
            return super().__getattribute__(value)
        except AttributeError as ex:
            return ex
            
    def __getattr__(self, value):
        '''此处的__getattr__不会被调用，因为AttributeError异常已经在__getattribute__中被捕获了'''
        return 'default'
    
at = AboutAttr("TEST")
print(at.name)
print(at.not_exised)  
getattr(at, 'name11')   #  该方法当属性存在的时候，返回属性值，不存在的时候返回异常  该方法实现的原理是调用当前对象__getattribuete__魔法函数        

TEST
'AboutAttr' object has no attribute 'not_exised'


AttributeError("'AboutAttr' object has no attribute 'name11'")

In [147]:
# __setattr__参见 doc文档说明  
# __setattr__调用的是当前实例的__dict__内置属性，返回值是一个字典类型，用于打印当前对象所含的属性（只有属性，没有实例的方法，实例的方法在类的__dict__中）  __init__创建实例属性的时候也是调用的__setattr__方法
class test():
    def __init__(self, a):
        self.a = a
    
    def fun(self):
        pass
aa = test(1) 
print(aa.__dict__)       # 只有属性，没有实例的方法，实例的方法在类的__dict__中
print(test.__dict__)

{'a': 1}
{'__module__': '__main__', '__init__': <function test.__init__ at 0x00000219AE8F89D8>, 'fun': <function test.fun at 0x00000219ADD8C488>, '__dict__': <attribute '__dict__' of 'test' objects>, '__weakref__': <attribute '__weakref__' of 'test' objects>, '__doc__': None}


In [25]:
'''
__setattr__(self, key, value)直接和存放实例属性的字典 __dict__联系，无论是初始化调用__init__()，还是在类中动态为实例添加属性，都会调用该魔法方法，
所以最好不要重写该方法，因为也会影响到__init__()中的初始化
'''
class F():
    def __init__(self, name):
        self.name = name
        
    def __getattribute__(self, item):
        print("called")
        return super().__getattribute__(item)
        
    def __setattr__(self, key, value):
        '''
        两点:
        第一点： 在使用super().__setattr__(key, value)的方式的时候，会使用用Python中原始的方式通过__setattr__为实例创建属性
        第二点： 因为第一点的原因，所以在想对实例属性定制化的时候，要将定制化的变化写在super().__setattr__(key, value)之前，这样再通过原生类的__setter__进行实例化属性的创建
        第三点： 使用原生类的__setattr__(key, value)并不会调用__dict__属性，因为__dict__也是属性，如果调用的话会触发__getattribute__方法，但是此处可以发现并没有调用__getattribute__
        第四点： 在__setattr__中对实例属性创建，一定要使用self.__dict__的方式，而不能self.k = value的字典赋值方式，否则会报最大递归错误
        第五点也是最重要的一点: 因为在执行初始化方法__init__()的时候，也是调用的__setattr__进行初始化的，如果对__setattr__做了修改，也会影响到__init__的初始化
        '''
        key = key + "_haha"
        super().__setattr__(key, value)  # 这是比较推荐的实例属性
        
#         print(key)
#         self.__dict__[key] = value
        
f = F("tmj")
print(f.name)
f.age = 12
print(f.age)
print(f.__dict__)
# 可以看到 无法调用name，提示当且类没有name属性，是因为初始化时__init__中也是调用的__setattr__将实例属性存储到__dict__中（python中，只要动态修改__dict__中的内筒，实际也就是修改的实例的属性，但只对当前实例有效，因为不同的实例不共享），
# 这里自定义的__setattr__中，对属性名就行了修改添加了_haha

called


AttributeError: 'F' object has no attribute 'name'

In [26]:
f = F("tmj")
print(f.name_haha) # 此时就可以调用了  所以一般不需要使用__setattr__方法

called
tmj


####  手动实现自定几个 实例属性 不能`被修改和创建`。见下例子。

In [28]:
# 手动实现自定几个实例属性不能被修改和创建。见下例子。
class F(object):  
    '''
    设置私有属性类的基类
    '''
    # 记录访问的次数，除去初始化的第一次
    access_count = 0
    def __setattr__(self, key, value):
        print("我在初始化的时候调用了")        
        try:
            if F.access_count <= 1:
                self.__dict__[key] = value
                F.access_count += 1
            elif key in self.privs: # 这里使用self调用privs，是因为实例对象可以使用类属性   
                # 之所以不会因为调用了当前类没有的privs类属性而报错，是因为__setattr__只有在__init__初始化（还必须有属性并进行赋值才调用__setattr__，
                # 因为初始化就是往__dict__字典中添值，添值才会调用__setattr__）和使用self.属性 = value 设置属性值的时候才会调用，当前类只数创建了
                raise AttributeError("该属性是私有属性，不能被创建或修改")
            else:
                self.__dict__[key] = value      # 这里保证了，即使创建的数量超过设置的2，只要不是privs中规定的私有属性，就可以创建    前面之所以设置2，是为了逻辑上当只有2个私有属性的时候能减少逻辑判断
        except AttributeError as ex:
            print(ex)

    
    # __getattribute__方法一般不需要重写
    def __getattribute__(self, item):
        return super().__getattribute__(item)   # __getattribute__必须有返回值，所以这里是必须是return 父类__getattribute__ 保证正常额功能，__getattribute__还关系到内置舒心__dict__ 见下例
        
    
    
    def __getattr__(self, item):
        str = '此属性无法创建，则无法读取'
        if item in self.privs:
            return str



class f(F):
      
    privs = ['name','age']

    def __init__(self):
        self.name = 'tom'
        self.age = '12'

f1 = f()                            ##  这里初始化的时候有两个属性，所以"我在初始化的时候调用了"打印了两次  __init__的时候是调用__setattr__执行的
f1.__dict__
print("-"*60)
f1.name = 'xiaoming'
print("-"*60)
print(f1.name1)
print("-"*60)

我在初始化的时候调用了
我在初始化的时候调用了
------------------------------------------------------------
我在初始化的时候调用了
该属性是私有属性，不能被创建或修改
------------------------------------------------------------
None
------------------------------------------------------------


In [ ]:
类的实例属性，是在创建实例化对象的时候才会初始化
类的实例方法，则是在类定义完时就初始化的了

In [24]:
class CC(object):
    def __getattribute__(self, item):
        return "dd"
    
c = CC()
print(c.__dict__)
print(CC.__dict__)

dd
{'__module__': '__main__', '__getattribute__': <function CC.__getattribute__ at 0x000001CD0FDB1B70>, '__dict__': <attribute '__dict__' of 'CC' objects>, '__weakref__': <attribute '__weakref__' of 'CC' objects>, '__doc__': None}


In [25]:
class c(object):
    def __getattribute__(self, item):
#         return "dd"
        return super().__getattribute__(item)
    
c = c()
print(c.__dict__)
print(CC.__dict__)

{}
{'__module__': '__main__', '__getattribute__': <function CC.__getattribute__ at 0x000001CD0FDB1B70>, '__dict__': <attribute '__dict__' of 'CC' objects>, '__weakref__': <attribute '__weakref__' of 'CC' objects>, '__doc__': None}


In [ ]:
1） 内置的数据类型没有__dict__属性

2） 每个类有自己的__dict__属性，就算存着继承关系，父类的__dict__ 并不会影响子类的__dict__

3） 对象也有自己的__dict__属性， 存储self.xxx 信息，父子类对象共用__dict__

In [16]:
# 8-3 属性描述符和属性查找过程      看描述器协议

'''
描述符类三个参数分别是  self 描述类的实例对象  instance为使用描述类的类的实例对象， 使用描述类的类对象

以上方法的参数一个不能少，可以使用其他变量名代替，但是个数不能少，返回值的位置固定，建议就用这几个变量做参数

注意：描述符类本身的实例对象进行属性操作并不会触发三个方法的执行代码。    

一个类实现了任意一个方法都可以称为属性描述符：
如果实现了__get__和__set__方法，称为数据描述符
如果只实现了__get__方法，称为非数据属性描述符
'''
class IntField:
    def __get__(self, instance, owner):
        print(self, instance, owner)
    def __set__(self, instance, value):
        print(self, instance, value)
    def __delete__(self, instance):
        print(self, instance)
        
class User:
    x = IntField()
    print(x)
    
u = User()       
u.x  
u.x = 1
print(u)
print(User)

<__main__.IntField object at 0x00000129E4C34668> <__main__.User object at 0x00000129E4C34710> <class '__main__.User'>
<__main__.IntField object at 0x00000129E4C34668> <__main__.User object at 0x00000129E4C34710> 1
<class '__main__.User'>


In [35]:
from datetime import date, datetime
import numbers
class IntField:
    #数据描述符
    def __get__(self, instance, owner):
        return self.value
    def __set__(self, instance, value):
        if not isinstance(value, numbers.Integral):
            raise ValueError("int value need")
        if value < 0:
            raise ValueError("positive value need")
        self.value = value   # 这里将数据存到描述符类实例对象的属性中 动态创建了一个属性
        # 也可以使用 self.__dict__['value'] = value
        
    def __delete__(self, instance):
        pass
    
class NonDataIntField:
    #非数据属性描述符
    def __get__(self, instance, owner):
        print("called")
        return self.value
    
class User:
#     age = IntField()   # 描述符类必须被定义为指派类的类属性，而不能在构造方法中。
    age = NonDataIntField()
    
if __name__ == "__main__":
    user = User()
    user.age = 2      #  可以发现此时虽然age是描述符类（是非数据描述符，所以当先使用 user.age = 2 这样的赋值语句的时候，就没有经过非数据描述符（因为没有__set__），此时age 就变成了User的实例对象的属性了，相当于动态添加，之后调用就__dict__[‘age’]）
    print(user.age)  # 此时调用的是User类调用 user.age = 2  时动态创建的属性age， 并没有调用 非数据描述符的 __get__
    print(user.__dict__)

# 说到底数据描述符类和非数据属性描述符类的区别在: 非数据描述符由于没有__set__，所以一旦当    user.age = 2   先出现，则非数据描述符类就不会被调用了，因为此时age这个非数据描述符类的实例对象已经变成了 指派类的实例对象属性了（相当于动态添加了）
# 因为下面的（2） 优先级高于 （3.1）

2
{'age': 2}


In [ ]:
属性的查找过程： 首先查找是否是类属性，然后在看是否是实例属性
而因为属性描述符类就是在User类中创建一个该属性描述符类的实例对象作为当前类的类属性

虽然__getattribute__和 __setattr__都只负责实例对象，但是当类属性被以  实例对象.类属性  的方式进行调用（因为类属性也可以被实例对象调用，但是如果进行修改的话，实际上是创建了一个实例属性）
但是在 实例对象.类属性 进行调用的时候，因为是以实例对象进行操作的，所以此时也会调用__getattribute__方法， 只限描述符的__get__，调用__get__的时候，会先经过__getattribute__

另外，当实例属性和类属性重名的时候，调用的是实例的属性  --  所以即使有描述符类，也不起作用，但是若此时类属性是数据描述符类，则会出现下面的情况一：

情况一：
class User:
    #     age = IntField()   # 描述符类必须被定义为指派类的类属性，而不能在构造方法中。
    name = IntField() # IntField()为描述符类

    def __init__(self):
        self.name = "tt"

像上面这样，实际上是__init__中调用的self.name 已经用的是描述符类的实例了，即此时就不是创建当前类的实例属性了 --- 这点和属性装饰器@property的用法相似（也会覆盖掉本该的赋值行为），实际上是因为@property内部的处理方式也是通过描述符装饰器实现的 
但是，若此时是非数据描述符类就没有关系，因为没有__set__，实际上这会就是创建的实例属性

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------

    def __init__(self):
        self.name = IntField()
        
如果像这样，在构造方法中进行定义，那么，描述符类就没有效果了，就是单纯的将一个类实例对象赋值给了一个属性

要注意：描述符类只对描述符类定义为指派类的类属性，这个类属性的操作有效果，其余的实例属性的操作，都是经过__getattribute__(self, item)魔法方法
数据描述符类:

非数据描述符类:
    
    
两者的实现方法的个数要求

没有__set__的时候，不能有__delete__吗？似乎是的，非数据描述符类中就不能有__delete__，因为没有 __set__，会报错： AttributeError: __set__
        
'''
如果user是某个类的实例，那么user.age（以及等价的getattr(user,’age’)）
首先调用__getattribute__。如果类定义了__getattr__方法，
那么在__getattribute__抛出 AttributeError 的时候就会调用到__getattr__，
而对于描述符(__get__）的调用，则是发生在__getattribute__内部的。   --------   确实，描述符类的__get__方法的调用，是在__getattribute__中进行的，打印user.age的时候会先进入__getattribute__内部，判断是描述符类，调用__get__
user = User(), 那么user.age 顺序如下：

（1）如果“age”是出现在User（因为属性描述符类就是在User类中创建一个该属性描述符类的实例对象作为当前类的类属性）或其基类的__dict__中（即'age'是基类的类属性）， 且age是data descriptor（数据描述符）， 
     那么调用其__get__方法, 否则    ---    即如果有数据描述符，则其优先级是最高的

（2）如果“age”出现在user的__dict__中（即‘age’是实例属性）， 那么直接返回 obj.__dict__[‘age’]， 否则

（3）如果“age”出现在User或其基类的__dict__中

（3.1）如果age是non-data descriptor （非数据描述符），那么调用其__get__方法， 否则    --- 这里面有种情况，如果是非数据描述符，但是没有先调用值（即没有要先执行__get__方法），
       而是对属性进行赋值，则由于非数据描述符类没有定义__set__方法，此时的self.user = 值， 实际上就是创建了一个实例属性，则此时之后的self.user也就是用的那个刚创建的实例属性
       但是通过 类.user 调用，还是使用的非数据描述符类的__get__进行访问的。
       
       这里原因：
           因为是将描述符类定义为类的属性，而且由于可以通过 self.类属性 的方式调用类属性的，所以当是数据描述符类的实例作为类属性的时候 self.类属性= 值，调用__set__， 打印self.类属性 调用__get__
           self.类属性 = 值，会调用__set__，就没有办法创建实例属性了（因为此时会调用描述符类的__set__方法，而一般的，由于只可以在类中通过 实例.类属性 的方式调用类属性，但是却不可以通过 self.类属性= 值 的方式修改类属性，
           如果这么用了，就会创建一个新的实例属性），所以一旦是为数据描述符类，则用永远没办法创建同名的实例属性，每次调用也都只是调用的数据描述符类     ，因为一个类的类属性是多个类实例对象共享的，
           （且一般情况下实例是不能修改类属性的，想执行修改的时候会自动创建同名的实例属性），所以数据描述符类的实例实际上用的同一个。 
           -------- 是不是可以根据这个特性，做点事
           但是若是非数据描述符类就不一样了，由于没有__set__，一旦先使用self.类属性= 值，由与没有__set__，也就不会调用__set__，这时候就会创建一个与类属性同名的实例属性，之后，由于当实例属性和类属性重名的时候，实例.类属性 调用的是实例的属性，
           单纯的打印 self.类属性 也不会调用__get__了，因为只有调用类属性的时候才会调用这个__get__，而现在无法通过self.类属性调用类属性了，只能调用的同名的刚创建的实例属性
           这中间是由于一个是类属性，而一个中间过程变为了实例属性，才造成了无法使用非数据描述符类
           

（3.2）返回 __dict__[‘age’]  --- 什么也不是，是一般的类属性

（4）如果User有__getattr__方法，调用__getattr__方法，否则

（5）抛出AttributeError

'''

所以即使是描述符类，使用__get__时也会经过__getattribute__，但只限在使用 self.类属性 或者 实例对象.类属性 的时候

In [55]:
# 案例。摄氏度华氏度转换
class Celsius:
    def __init__(self, value=26.0):
        self.value = float(value)
    
    def __get__(self, instance, owner):
        return self.value
    
    def __set__(self, instance, value):
        self.value = float(value)
    
class Fahrenheit:
    def __get__(self, instance, owner):
        return instance.cel * 1.8 + 32
    
    def __set__(self, instance, value):
        instance.cel = (float(value) - 32) / 1.8
        
class Temperature:
    cel = Celsius()
    fah = Fahrenheit()
    
temp = Temperature()
temp.cel = 30
print(temp.fah)
temp.fah = 100
print(temp.cel)


86.0
37.77777777777778


In [112]:
# 8-4 __new__和__init__的区别  参见笔记
'''
使用 类名() 创建 实例对象的时候，解释器首先会调用__new__方法为 实例对象 分配空间
在使用__new__方法为实例对象分配空间之前，Python解释器已经完成了类的创建，即已经为类分配了内存地址。这个就是__new__(cls)中cls中接受的是当前类。
__new__是一个由object 基类提供的内置静态方法，主要作用有两个:
    1.在内存中为 实例对象 分配空间
    2.返回 实例对象 的引用
    3.创建实例对象时传进的实例属性会先经过__new__(cls, *args, **kwargs)，再由__new__传递给__init__
为什么会是静态方法：
    因为静态方法的调用不需要创建类的实例对象，就可以通过 类.静态方法 的方式来进行调用，类对象和实例对象都可以调用静态方法。 
    静态方法的所有参数都是都是要手动传递的。
    
Python的解释器获得当前类实例对象的引用之后，将引用作为第一个参数，传递给__init__方法
创建对象的过程：
使用 类名() 创建实例对象的时候，解释器首先会调用（此类中由基类object类继承下来的）__new__方法为对象 分配空间然后返回实例对象的引用，Python的解释器获得实例对象的引用之后，将引用作为第一个参数，传递给__init__方法，__init__方法进行实例对象的属性值
初始化，然后在__init__(self,x)中通过调用__setattr__(self, key, value) 为实例创建属性


使用时注意:重写__new__方法时，一定要调用父类的__new__方法获得创建的实例并进行返回，否则解释器无法获得创建的实例对象（的引用）
'''

class User:
    def __new__(cls, *args, **kwargs):    
        print(args)
        print(kwargs)
        print (" in new ")
        return super().__new__(cls)   # 要return 返回生成的实例对象内存地址  object的__new__只有一个cls参数
    def __init__(self, name):
        print (" in init")

#new 是用来控制实例对象的生成过程， 在实例对象生成之前
#init是用来完善实例对象的
#如果new方法不返回对象， 则不会调用init函数
if __name__ == "__main__":
    user = User("bobby")    # 传递的参数经过 __new__ 所以我们可以在这里面进行定制化
    print("---"*30)
    user = User(name = "bobby") 


('bobby',)
{}
 in new 
 in init
------------------------------------------------------------------------------------------
()
{'name': 'bobby'}
 in new 
 in init


In [3]:
# 应用  单例模式
"""
实现单例模式：
"""
class SingleCase(object):
    instance = None
    def __new__(cls, *args, **kwargs): 
        '''
        这里是一定要加上*args, **kwargs参数，用来接收调用__init__时，要接受的参数
        就是__new__方法会接受定义实例的时候传递的参数，
        这里的参数是先经过__new__方法的，再有__new__传递给__init__
        '''
        if not cls.instance :
            cls.instance = super(SingleCase, cls).__new__(cls)
        return cls.instance

    def __init__(self, name, age):
        print("called")
        self.name = name
        self.age = age
'''
虽然传递了两次 属性的值
但是显示的是最新的属性值
是因为使用的同一个类实例，每个类实例都只有一个__dict__，所以修改的是同一个__dict__
'''
single_case1 = SingleCase('tmj',12)
print(single_case1.name)
single_case2 = SingleCase('skylor', 14)
print(single_case1.name)
print(single_case2.name) 

called
tmj
called
skylor
skylor


In [ ]:
'''
python中
类调用类方法（@classmethod）不需要传递参数cls
类调用实例方法，需要手动传递实例对象
实例可以调用类方法（@classmethod）不需要传递参数cls
实例调用实例方法，不需要传递参数self

静态方法：使用@staticmethod修饰，理论上静态方法是不会调用类以及实例的属性或者方法
只有这样才能在没有创建实例对象的时候进行调用
一般通过类对象迪用静态方法
实例对象也是可以调用静态方法的

总结就是，实例对象可以调用各种方法

另外，在实例没有某个属性的时候，若有同名的类属性，则就会调用类属性，如果此时再进行赋值操作，则不是修改类属性，而是动态为当前实例对象创建一个实例属性
        但是切记，类不能调用实例属性
'''

In [5]:
# 8-5 自定义元类
# type创建类， 是object类的子类， object是所有类的基类，所有类都是type类的实例化对象   所以类也是对象


def create_class(name):
    '''函数，动态创建类'''
    if name == "user":
        class User:
            def __str__(self):
                return "user"
        return User
    elif name == "company":
        class Company:
            def __str__(self):
                return "company"
        return Company

if __name__ == "__main__":
    MyClass = create_class("user")
    my_obj = MyClass()
    print(type(my_obj))
    print(my_obj)

<class '__main__.create_class.<locals>.User'>
user


In [ ]:
# type的3种用法：
# type(object_or_name, bases, dict)  动态创建类
# type(object) -> the object's type  得到类型
# type(name, bases, dict) -> a new type 

In [10]:
# 使用type动态创建类
class Baseclass:
    def answer(self):
        return "i am baseclass"
    
def say(self):
#     return "i am user"
    return self.name # 调用的时候，如果类有同名的属性，则调用类的同名属性
User = type("User", (Baseclass, ), {"name":"skylor", "say":say})     #三个参数 ： 类的名称， 类的基类tuple类型， 类属性和类方法实例方法 dict类型  
my_obj = User()
print(my_obj)
print(my_obj.__dict__)
print(User.__dict__) # 可见动态创建类的时候 定义的属性是类的属性
print(my_obj.say())
print(my_obj.answer())

{}
{'name': 'skylor', 'say': <function say at 0x0000029BA5DD7F28>, '__module__': '__main__', '__doc__': None}
skylor
i am baseclass


In [ ]:
https://www.cnblogs.com/chvv/p/9950837.html

python中的__new__方法
1.创建类时先执行type的__init__方法,
2.当一个类实例化时(创建一个对象)执行type的__call__方法，__call__方法的返回值就是实例化的对象
　　　　__call__内部调用
　　　　　　类.__new__方法，创建一个对象
　　　　　　类.__init__方法，初始化对象

实例化对象是谁取决于__new__方法,__new__返回什么就是什么

　__new__() 方法的特性：
        __new__() 方法是在类准备将自身实例化时调用。
        __new__() 方法始终都是类的静态方法，即使没有被加上静态方法装饰器

In [29]:
# 什么是元类，元类是创建类的类  对象由class创建，而class也是对象,由type创建 所以type是也是一个元类
'''
    实际上，以下 MetaClass 自定义元类中的cls self 的标准写法为 mcs cls
'''
"""
一般在使用中很少使用type来创建类，一般我们都是通过自定义元类的方式来创建类
"""
class MetaClass(type):   # 一个类继承了type （必须要继承type才是元类，因为要调用type类的__new__方法完成类的创建（因为类比用类的__new__创建实例对象，type的__new__创建的也是type的实例对象，而type类的实例对象就是各种类） ） 就是一个元类 
                         # 自定义元类 能够控制 metaclass等于该 自定义元类 的 类 实例化的过程     
    '''
        除了其他类在使用metaclass来使用元类，也可以直接传参生成元类的实例化对象，该对象就是一个类
        用当前自定义元类生成实例化对象--一个新类时候，注意以下几项：
            1. 生成实例对象--新类时，要接收的参数一般由三个，类比普通类创建实例对象的方式，接受的参数由__init__最终接收，用于创建自定义元类实例对象--一个新的类，和普通类一样，中间会经过__new__
               三个参数和使用type(object_or_name, bases, dict)动态创建类传递的参数一样，为 object_or_name:str 创建的类的名字 bases:tuple 创建的类继承的父类，可以是多个 dict:dict 创建的类的属性和方法
            2. return super().__new__(cls, *args, **kwargs)  # metaclass中的__new__要接受*args, **kwargs这两个参数  （这点和普通的类创建实例对象不同）
    '''
    def __new__(cls, *args, **kwargs):  # cls 为自定义元类MetaClass， args是 ('User',(),{'__module__':'__main__','__qualname__':'User'})就是type()动态创建类时的参数   。、kwargs是{}      
        '''
        __new__方法的标准参数是 __new__(mcls, name, bases, namespace)
        ''' 
        print(cls)
        print(args)   # 控制了类的创建， 接收到类的没名字，基类，方法 属性
        print(kwargs)
        print("---"*30)
        return super().__new__(cls, *args, **kwargs)  # metaclass中的__new__要接受*args, **kwargs这两个参数              __new__方法返回类的引用（即创建了类这个类对象），然后类对象就可以通过my_obj = User(name="bobby")创建一个实例了
        
    
    def __init__(self, *args, **kwargs):
        self.class_attribute = "我是类的属性" # 接受了 type(object_or_name, bases, dict) 传递过来的关于创建的类的设置，  在__init__还可以手动创建一些当前类的其他的属性和方法，（把类方法当做方法）
#         super().__init__(*args, **kwargs)   # 这句是没有用的
    
    def __getattr__(self, item):
        '''
            元类就是控制类的创建的类，所以可以进行类的调控
            这里使用__getattr__就能控制类的属性，像在实现类中的__getattr__可以调控实例类的实例对象属性一样-----因为类是元类的实例对象
        '''
        return "当前类没有该类属性"
    
#     def say(self):
#         '''
#             创建了，但是为什么使用User.__dict__中没有该方法呢，不是应该是类的方法吗
#         '''
#         print("我是User类的类方法")
    
class User(metaclass=MetaClass):  # 创建该类时，先调用MetaClass
    '''类比上面的__new__，可以发现 自定义元类就是控制了类的创建，而定义__new__就是控制类的实例化对象的创建'''
    def __new__(cls, *args, **kwargs):
        print(cls)
        print(args)
        print(kwargs)
        print("---"*30)
        
        return super().__new__(cls) # 使用 类名() 创建实例对象的时候，解释器首先会调用（此类中由基类object类继承下来的）__new__方法为对象 分配空间然后返回实例对象的引用，，将引用作为第一个参数，传递给__init__方法进行实例对象的初始化，定义实例属性
    
    def __init__(self, name):
        self.name = name
                                             
#     def __str__(self):
#         return "user"     # 不设置的返回值为<__main__.User object at 0x000001C282F769B0>

if __name__ == "__main__":
    my_obj = User(name="bobby")
    print(my_obj)
    print("---"*30)
    print(type(User))
    print("---"*40)
    print(User.dddddd)    # 调用类的属性不存在的时候使用的是 元类的__getattr__，类比 某个类的实例对象，类就是元类的实例对象 
    print(User.class_attribute)   # 使用 元类的__init__实现
    print(User.__dict__)
#     User.say()



#     Test = MetaClass('Test',(),{'__module__':'__main__','__qualname__':'Test'})   # 就像使用type动态创建类那样，具体去看type的使用    这样使用的好处 就是相对于使用type，对类的定制化更加高了
#     print("---"*40)
#     print(Test.__dict__)
    
    
    


<class '__main__.MetaClass'>
('User', (), {'__module__': '__main__', '__qualname__': 'User', '__doc__': '类比上面的__new__，可以发现 自定义元类就是控制了类的创建，而定义__new__就是控制类的实例化对象的创建', '__new__': <function User.__new__ at 0x0000029BA5F81510>, '__init__': <function User.__init__ at 0x0000029BA5F81598>, '__classcell__': <cell at 0x0000029BA5B2E6D8: empty>})
{}
------------------------------------------------------------------------------------------
<class '__main__.User'>
()
{'name': 'bobby'}
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
<class '__main__.MetaClass'>
------------------------------------------------------------------------------------------------------------------------
当前类没有该类属性
我是类的属性
{'__module__': '__main__', '__doc__': '类比上面的__new__，可以发现 自定义元类就是控制了类的创建，而定义__new__就是控制类的实例化对象的创建', '__new__': <staticmethod object at 0x0000029BA5FC4828>, '__init__': <functi

In [25]:
class bbbbb():
    @classmethod
    def say(cls):
        print("fas")
        
        
b = bbbbb()
print(bbbbb.__dict__)

{'__module__': '__main__', 'say': <classmethod object at 0x0000029BA5F79940>, '__dict__': <attribute '__dict__' of 'bbbbb' objects>, '__weakref__': <attribute '__weakref__' of 'bbbbb' objects>, '__doc__': None}


In [49]:
class a():
    def __new__(cls, *args, **kwargs):
        return super().__new__(cls)
    
    def __init__(self, *args, **kwargs):
        self.aa = 33
        
        
b = a(11, v= 1)  
b.__dict__

{'aa': 33}

In [ ]:
'''
Python中类创建的时候，会首先寻找metaclass属性，找到则通过metaclass定义的自定义元类去创建User类   --- 若一个类是继承自一个类，那么首先会寻找自己有没有metaclass，然后在看基类有没有metaclass
没有metaclass的类会在创建的时候调用type去创建一个User的类对象    --  所以metaclass的优先级非常的高

# 一旦某个类有metaclass 或者其父类含有metaclass  则这些类的类型都是 metaclass指定的类， 即是通过该类创建了这些类。 ---- 只要继承关系上某个类有metaclass，则就会从该类开始，以及他的子类都会通过metaclass进行创建
# 一旦一个类有metaclass 或者其父类含有metaclass  则都会调用自定义元类来创建
# 当是父类User含有metaclass的时候，则是父类User先调用自定义元类创建父类，然后调用自定义元类创建子类T，且创建的时候子类有的('T', (<class '__main__.User'>,), {'__module__': '__main__', '__qualname__': 'T'})其父类(<class '__main__.User'>,)
# 即使用metaclass创建了子类T，且子类同时会继承父类User
'''

In [ ]:
我们在指定一个类的元类是ABCMeta的时候，就能帮我们自动检查是否有未实现的抽象方法的时候，其实就是通过自定义元类ABCMeta在其__new__的检测其子类在定义的时候抽象方法是否实现了这些抽象方法
因为在实例化的时候，类首先会看是否有metaclass属性，没有则看其继承的父类是否有metaclass，若都没有，才会调用type这个元类去创建类

In [49]:
# 如何实现在初始化的时候就抛出异常呢（当没有实现抽象基类的方法的时候）
import abc

class CacheBase(metaclass=abc.ABCMeta):     
    
    @abc.abstractmethod
    def get(self, key):
        pass # 此处就可以直接pass了，不需要raise NotImplementedError，  因为使用了abc，这样当一个类继承该抽象基类时没有实现其抽象方法的时候，就会报错
                                  # 相当于之前是手动加保险，用raise手动抛出异常，这里我们借助abc模块，所以即不用手动在写出来了。
    @abc.abstractmethod
    def set(self, key, value):
        pass
    
class RedisCache(CacheBase):
    def get(self, key):
        pass
    
    def set(self, key, value):     # python 中只要求方法名相同
        pass

redis_cache = RedisCache()  # 这样直接在初始化阶段就会告知创建 RedisCache(CacheBase)类的时候要实现的抽象方法 TypeError: Can't instantiate abstract class RedisCache with abstract methods get, set 
print(type(redis_cache))

<class '__main__.RedisCache'>


In [ ]:
知识点：重载：
'''
Python中没有重载，即不可能实现说：一个类中有两个方法的方法名相同，但是参数不同这种用法
    java中的重载的定义：重载(overloading) 是在一个类里面，方法名字相同，而参数不同。返回类型可以相同也可以不同。
    但是Python中完全就不需要重载：
        1. 参数不同： 可以通过缺省参数（可变参数）*args 和 **kwargs，也可以通过传递列表这种可变序列类型，来完成。
        2. 类型问题： Python没有类型限制
    https://www.cnblogs.com/erbaodabao0611/p/7490439.html：
    简单来说，Python中为什么不需要重载，重载要解决的是参数类型和参数个数的问题，对于类型，python不像是c语言整型要写int，字符串要写str,,,这些python都不需要。
    那么需要解决的就是传递参数个数问题，此时python可以传递列表呀，字典呀，可以使用*arg和**args呀，所以python根本不需要重载。
    
Python具有重载的思想却没有重载的概念，由于本身的特性，不需要重载。
python提供@singledispatch实现重载的功能 ------- 流畅的Python
https://www.jb51.net/article/150579.htm
https://www.cnblogs.com/zhuangliu/p/10851268.html
'''

In [ ]:
ABCMeta 的实现（Python3.5）：
def __new__(mcls, name, bases, namespace):
        cls = super().__new__(mcls, name, bases, namespace)
        # Compute set of abstract method names
        abstracts = {name
                     for name, value in namespace.items()
                     if getattr(value, "__isabstractmethod__", False)}
        for base in bases:
            for name in getattr(base, "__abstractmethods__", set()):
                value = getattr(cls, name, None)
                if getattr(value, "__isabstractmethod__", False):
                    abstracts.add(name)
        cls.__abstractmethods__ = frozenset(abstracts)
        # Set up inheritance registry
        cls._abc_registry = WeakSet()
        cls._abc_cache = WeakSet()
        cls._abc_negative_cache = WeakSet()
        cls._abc_negative_cache_version = ABCMeta._abc_invalidation_counter
        return cls
# 会进行一系列的检查

In [18]:
# 8-6 通过元类实现orm   以Django中的ORM方式实现

'''
需求:
orm  将类映射到数据库中的一种表 对类进行操作，就可以将数据写到数据库的表中

描述符类CharField数据库的字段类型char ，db_colum指明在数据库中所属的列 ,max_length 字段最大长度
描述符类IntField, 

    单独的内部类 Meta 指明数据库表的名称   而不希望在定义数据库的类中实现    不定义该Meta的时候，默认使用类型User的小写来实现数据库表的名称

class User:  # 不定义内部类Meta的时候，默认使用类型User的小写来实现数据库表的名称
    name = CharField(db_colum='' ,max_length=10)  # 字段名
    age = IntField(db_colum='',min_value=0, max_value=10)  # 字段名
    
    class Meta：
        # 内部类，创建数据库表名
        db_table = "数据库表的名称"  
        abstract 
        

if __name__ = "__main__":
    # 表的创建
    user = User()
    # 表数据的创建   使用的是描述符类的特性  __set__
    user.name = "xiaoming"   
    user.age = 28                 # 希望在设置值时，能够完成属性赋值报错的功能，  属性描述符完成
    user.save()
    
    
    # 内部受保护的属性 使用_  一种约定
'''
import numbers

class Field:
    '''
    作为IntField和CharField的父类，用于在下面的判断中能够将这两种数据通过isinstance判断取出
    '''
    pass

class IntField(Field):
    # 数据描述符
    def __init__(self, db_column=None, min_value=None, max_value=None):
        self._value = None                   # 这里_value 作为 user.age = 28接受值28的属性，可以是动态生成的，但是为了保证如果先调用__get__能有值，所以设置在这里。 也是可以在__get__中进行参数检查是否存在的，但是逻辑上复杂
        self.min_value = min_value
        self.max_value = max_value
        self.db_column = db_column
        # 以下是大量的数据检查
        if min_value is not None:
            if not isinstance(min_value, numbers.Integral):
                raise ValueError("min_value must be int")
            elif min_value < 0:
                raise ValueError("min_value must be positive int")
        if max_value is not None:
            if not isinstance(max_value, numbers.Integral):
                raise ValueError("max_value must be int")
            elif max_value < 0:
                raise ValueError("max_value must be positive int")
        if min_value is not None and max_value is not None:
            if min_value > max_value:
                raise ValueError("min_value must be smaller than max_value")
    
    def __get__(self, instance, owner):
        return self._value
                
    def __set__(self, instance, value):
        if not isinstance(value, numbers.Integral):
            raise ValueError("int value need")
        if value < self.min_value or value > self.max_value:
            raise ValueError("value must between min_value and max_value")
        self._value = value
        
class CharField(Field):
    def __init__(self, db_column=None, max_length=None):
        self._value = None
        self.db_column = db_column
        if max_length is None:
            raise ValueError("you must spcify max_lenth for charfiled")
        self.max_length = max_length
    
    def __get__(self, instance, owner):
        return self._value
      
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError("string value need")
        if len(value) > self.max_length:
            raise ValueError("value len excess len of max_length")
        self._value = value
        
class ModelMetaClass(type): 
    '''数据库模型元类
    模仿ABCMeta的写法：
    __new__(mcls, name, bases, namespace)
    可以这么写，因为我们不是手动创建一个该元类的实例对象
    而是在创建类的时候，通过 metaclass = 自定义元类 参数，告诉解释器，由解释器来进行掉用
    则那三个参数就会按序传递给 __new__(mcls, name, bases, namespace)
    所以当是交个解释器使用时，就不需要防止参数过多之类的而使用 *args和**kwarg 可变参数参数
    
    实际上当解释调用的时候，只会传递三个参数 name, bases, namespace
    实际上下面的new只需要__new__(cls, name, bases, attrs)即可，attrs是个字典 namespace
    然后return super().__new__(cls, name, bases, attrs) 也只需要三个参数就行了
    这里的**kwargs永远是没有内容的
    
    但这里因为type的__new__是__new__(*args, **kwargs)， 所以没有问题，不论传多少都会被接收
    
    '''
    def __new__(cls, name, bases, attrs, **kwargs):    
        if name == "BaseModel":
            return super().__new__(cls, name, bases, attrs, **kwargs)  # 序列拆包   
        
        fields = {}  # 记录数据表相关的列
        for key, value in  attrs.items():
            if isinstance(value, Field):  # 将User的类属性（字段名对象拿出来，即数据表相关的列）
                fields[key] = value
        attrs_meta = attrs.get("Meta", None) # 看是否定义了内部类，将内部类对象方进attrs_meta中
        _meta = {}  
        db_table = name.lower()
        if attrs_meta is not None:
            table = getattr(attrs_meta, "db_table", None)  # getattr() 函数用于返回一个对象属性值(类对象，实例对象都可以)。看内部类对象是否有 db_table 属性
            if table is not None:
                db_table = table  # 覆盖之前的db_table = name.lower()- 使用类名的小写作为表名
        _meta["db_table"] = db_table   
        attrs["_meta"] = _meta      # 添加了数据库表名
        attrs["fields"] = fields   # 往User类中添加了 数据库包含的列名
        del attrs["Meta"]         # 只是为了得到db_table数据库表名，获得后将User类中创建了Meta类删除
        return super().__new__(cls, name, bases, attrs, **kwargs)  # 调用type类的__new__，完成User类的创建   至此，通过自定义元类的方式 往User中动态添加了数据库表名，数据库的列名（即数据表字段）

class BaseModel(metaclass=ModelMetaClass):
    '''
    简化User的定义，简化使用
    完成User()类的其他一些细节功能，如__init__ 和save()方法
    '''
    def __init__(self, *args, **kwargs):   # 实现BaseModel是为了定义一个__init__方法，接受用户直接在创建实例对象的时候传递的参数如 user = User(name='',age="")
        '''
        实际也是做了映射，看似是创建属性，实际上是调用的User的描述符类，去给描述符类的属性创建值
        '''
        for key, value in kwargs.items():
            setattr(self, key, value)     
        return super().__init__()  # 重写了一些重要内置的方法的时候（如__init__,__new__,__getattribute__），一定要继承父类的同类方法，因为这些方法关系到类功能的完整   
                                   # 这里的super()得到的是object类  此外 object的__init__没有参数，__new__因为是静态方法，除了cls也没有其他的参数，      
        
    def save(self):
        '''拼凑sql字符串，实现保存功能'''
        fields = []
        values = []
        for key, value in self.fields.items():
            db_column = value.db_column  # 字段名
            if db_column is None:
                db_column = key.lower()  # 没有设置db_colum时，使用的是User类属性名（即字段名）的小写
            fields.append(db_column)
            value = getattr(self, key)  # self.key  调用的是描述符类的__get__方法
            print(value)
            values.append(str(value))   # 要变成string类型，因为是

        sql = "insert {db_table}({fields}) value({values})".format(db_table=self._meta["db_table"],
                                                                   fields=",".join(fields), values=",".join(values))
        
class User(BaseModel):   # 自身没有metaclass的时候，会在父类中查找metaclass，若父类有metaclass,则使用该metacalss进行创建类
    '''
    实际上要想实现将类User能够映射到数据表还需要内部很多的操作，class User实际还有很多的属性要做，所以在创建User的时候，我们可以注册进自己需要的很多东西到User中 
    通过自定义元类ModelMetaClass往我们的类中注入（动态添加）一些属性，方法之类的，供一些特有的方法使用这些属性
    
    这里通过自定义元类 动态 添加了数据库表名，数据库的列（字段名和值）   
    然后将剩下的数据库操作，等都放到父类BaseModel中，在父类中设置__init__，save由子类User调用
    而子类User中只留最简单的定义设置
    
    
    整个过程：
    由User类到元类 再到User类，再到父类的save方法 
    
    '''
    name = CharField(db_column="name", max_length=10)
    age = IntField(db_column="age", min_value=1, max_value=100)

    class Meta:
        db_table = "user"


if __name__ == "__main__":
    user = User(name="bobby", age=28)   # 由于BaseModel的__init__方法设置，这里要求必须是以关键字参数的形式传递参数
#     user.name = "bobby"
#     user.age = 28
    user.save()
    print(User.__dict__)

bobby
28
{'__module__': '__main__', '__doc__': '\n    实际上要想实现将类User能够映射到数据表还需要内部很多的操作，class User实际还有很多的属性要做，所以在创建User的时候，我们可以注册进自己需要的很多东西到User中 \n    通过自定义元类ModelMetaClass往我们的类中注入（动态添加）一些属性，方法之类的，供一些特有的方法使用这些属性\n    \n    这里通过自定义元类动态添加了数据库表名，数据库的列（字段名和值）   由本类到元类 再到本类，再到父类的save方法 \n    \n    ', 'name': <__main__.CharField object at 0x000001E9A897D1D0>, 'age': <__main__.IntField object at 0x000001E9A897D0B8>, '_meta': {'db_table': 'user'}, 'fields': {'name': <__main__.CharField object at 0x000001E9A897D1D0>, 'age': <__main__.IntField object at 0x000001E9A897D0B8>}}


In [ ]:
注意一点： 自定义元类的__new__方法用于控制类的创建，可以对类的属性，方法，继承的类指定进行设置   ，而 类的__new__方法用于控制其实例对象的创建，可以对实例的属性进行一些设置
自定义元类的__new__接受的参数是type()创建类时的参数， 而类的__new__方法接受的是实例属性给的值
自定义元类的__new__接受参数中有类的方法，包括__init__，__new__（注意，这几个方法定义类的时候没写的时候是不会自动产生传进的）这些等等，这些方法只是有了内存空间，但是还没有被使用
等到自定义元类的__new__调用结束，并返回了type.__new__，则我们的类就创建完成了，此时类在调用类自己的__new__方法 为实例对象分配空间，然后返回实例对象的引用和定义时接受到的参数（即属性的值），
Python的解释器获得实例对象的引用之后，将引用作为第一个参数，传递给__init__方法，__init__方法进行实例对象的初始化，然后定义实例属性（由类自己的__new__方法接收的，-- 这里__new__是可以接受值进行修改的）

In [ ]:
getattr() 函数用于返回一个对象属性值(类对象，实例对象都可以)
当对象是实例对象的时候，实际调用的是类内部的__getattribute__方法

In [3]:
class a():
    def __init__(self,b):
        self.b = b
        
    def pp(self):
        print(self.a)
        
class b(a):        
    def __init__(self, a,b):
        self.a = a
        super().__init__(b)
        
bb = b(3,3)
bb.pp()
print(b.__dict__)
print(a.__dict__)

print(bb.__dict__)

3
{'__module__': '__main__', '__init__': <function b.__init__ at 0x000001E9A84E31E0>, '__doc__': None}
{'__module__': '__main__', '__init__': <function a.__init__ at 0x000001E9A84E3400>, 'pp': <function a.pp at 0x000001E9A84E3158>, '__dict__': <attribute '__dict__' of 'a' objects>, '__weakref__': <attribute '__weakref__' of 'a' objects>, '__doc__': None}
{'a': 3, 'b': 3}


In [ ]:
可见，父类和子类都有自己的__dict__，且两者是不同享的
但是子类的实例对象拥有父类实例的属性__dict__

In [ ]:
所有的类都是object类的子类，即object类是所有类的基类，包括type
然后所有的类都是type类的实例对象，包括object

In [ ]:
**kwargs在定义函数的时候做参数，是指的可变参数
但是当调用函数的时候，若调用函数，然后传递了一个 **变量， 这里的**变量是作为解包用的 见例二 

In [4]:
# 例一:
def internal_function(**a):
    print(a)


def func():
    kwargs = {'a': 1}
    internal_function(**kwargs)  # 此处确实是用的kwargs解包，不过得到的是关键字参数的形式，如 a = 1这样，所以也要对应函数调用的时候，用**a 可变参数类型

func()

{'a': 1}


In [7]:
# 例二：
def internal_function(a):
    print(a)

'''
因为kwargs = {'a': 1} 直接**kwargs 
'''
def func():
    kwargs = {'a': 1}
    '''
    此处的**kwargs 是解包，得到的是 a = 1 ,类似这样，但是这是不能打印的，因为a是实实在在的变量
    只能用参数是同名的函数来进行接受如这里的internal_function的参数是a，能正常运行
    但是如参数是b或其他非a的参数，则就会报错
    
    所以一般像例一那样使用，这样例一能接受到完整的字典类型
    '''
    internal_function(**kwargs)

func()

1
